In [156]:
from env import get_connection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from math import sqrt
from scipy import stats
from scipy.stats import pearsonr, spearmanr

import os

In [157]:
def df_to_csv(df):
    try:
        df.to_csv(filename, index=False, mode='x')
    except FileExistsError:
        df.to_csv(filename,index=False)

In [158]:
def offline_lesson_kernel_restart():
    exists = os.path.isfile(filename)
    if exists:
        df = pd.read_csv(filename)
        
        return df
    else:
        #Define query
        query = '''
                select * 
                from `logs`
                JOIN cohorts ON logs.cohort_id=cohorts.id;
                '''
        #Define url
        url = get_connection('curriculum_logs')
        
        #Read data from SQL server
        df = pd.read_sql(query, url)
        
        #Cache
        df_to_csv(df)
        
        return df
filename = "curriculum_logs.csv"
df = offline_lesson_kernel_restart()
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [159]:
df['fixed_date']= df.date + ' ' + df.time

In [160]:
df.drop(columns= ['id', 'slack', 'deleted_at', 'date', 'time'], inplace= True)

In [161]:
df

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,fixed_date
0,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-26 09:55:03
1,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-26 09:56:02
2,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-26 09:56:05
3,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-26 09:56:06
4,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,2018-01-26 09:56:24
...,...,...,...,...,...,...,...,...,...,...,...
847325,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,2021-04-21 16:41:51
847326,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,2021-04-21 16:42:02
847327,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,2021-04-21 16:42:09
847328,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,2021-04-21 16:44:37


In [162]:
df.program_id.value_counts()

2    713365
3    103412
1     30548
4         5
Name: program_id, dtype: int64

In [163]:
df.set_index(['fixed_date'], inplace= True)
df

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
fixed_date,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2
...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2


In [164]:
df['data']= df['program_id']== 3
df['web']= df['program_id']== 2
df['php']= df['program_id']== 1
df['front_end']= df['program_id']== 4

In [165]:
df

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,data,web,php,front_end
fixed_date,,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,False,False,True,False
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,False,False,True,False
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,False,False,True,False
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,False,False,True,False
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,False,True,False,False
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,False,True,False,False
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,False,True,False,False


 # Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [133]:
def q_two(df):
    weird_df= df.groupby(['cohort_id', 'path']).agg('count')
    weird_df.sort_values(by= 'user_id', ascending= False)
    weird_df= weird_df.reset_index()
    weird_df= weird_df[weird_df.path != '/']
    count_df= weird_df[['cohort_id', 'path', 'ip']]
    count_df.sort_values(by= ['cohort_id', 'ip'], ascending= [False, False])
    new_df= count_df.sort_values(by= 'ip', ascending= False).groupby('cohort_id').nth(0)
    return new_df

In [128]:
new_df= Q_two(df)
new_df

,path,ip
cohort_id,,
1.0,javascript-i,294
2.0,content/php_ii/command-line,6
4.0,mkdocs/search_index.json,1
6.0,javascript-ii/es6,10
7.0,content/html-css,29
8.0,java-iii,57
9.0,content/html-css,2
11.0,html-css,12
12.0,content/html-css,19


In [134]:
def q_seven(df):
    least_df= df.groupby(['cohort_id', 'path']).agg('count')
    least_df.sort_values(by= 'user_id', ascending= False)
    least_df= least_df.reset_index()
    least_df= least_df[least_df.path != '/']
    new_least_df= weird_df[['cohort_id', 'path', 'ip']]
    new_least_df.sort_values(by= ['cohort_id', 'ip'], ascending= [False, False])
    newer_df= new_least_df.sort_values(by= 'ip', ascending= False).groupby('cohort_id').nth(-1)
    return newer_df

In [135]:
newer_df= q_seven(df)

In [136]:
newer_df

,path,ip
cohort_id,,
1.0,2.0_Intro_Stats,1
2.0,prework/fundamentals/loop,1
4.0,prework/versioning/github,1
6.0,11._DistributedML,1
7.0,content/examples/gitbook/images/favicon.ico,1
8.0,content/php_iv/php-with-mysql/gitbook/images/f...,1
9.0,content/html-css/introduction.html,1
11.0,content/php_i/gitbook/images/favicon.ico,1
12.0,prework/cli,1


In [205]:
def q_seven(df):
    least_df= df.groupby(['cohort_id', 'path']).agg('count')
    least_df.sort_values(by= 'user_id', ascending= False)
    least_df= least_df.reset_index()
    least_df= least_df[least_df.path != '/']
    new_least_df= weird_df[['cohort_id', 'path', 'ip']]
    new_least_df.sort_values(by= ['cohort_id', 'ip'], ascending= [False, False])
    newer_df= new_least_df.sort_values(by= 'ip', ascending= False).groupby('cohort_id').nth(-1)
    newer_df.rename(columns= {'ip': 'count'}, inplace= True)
    return newer_df

In [206]:
dffff= q_seven(df)
dffff

,path,count
cohort_id,,
1.0,2.0_Intro_Stats,1
2.0,prework/fundamentals/loop,1
4.0,prework/versioning/github,1
6.0,11._DistributedML,1
7.0,content/examples/gitbook/images/favicon.ico,1
8.0,content/php_iv/php-with-mysql/gitbook/images/f...,1
9.0,content/html-css/introduction.html,1
11.0,content/php_i/gitbook/images/favicon.ico,1
12.0,prework/cli,1


In [203]:
def q_seven_two(df):
    least_df= df.groupby(['program_id', 'path']).agg('count')
    least_df.sort_values(by= 'user_id', ascending= False)
    least_df= least_df.reset_index()
    least_df= least_df[least_df.path != '/']
    new_least_df= least_df[['program_id', 'path', 'ip']]
    new_least_df.sort_values(by= ['program_id', 'ip'], ascending= [False, False])
    newer_df= new_least_df.sort_values(by= 'ip', ascending= False).groupby('program_id').nth([-1, -2, -3])
    newer_df.rename(columns= {'ip': 'count'}, inplace= True)
    return newer_df

In [204]:
dff= q_seven_two(df)
dff

,path,count
program_id,,
1,cohorts/24/capstone-all,1
1,slides,1
1,cohorts/24/grades,1
2,classification/user-defined-functions,1
2,cheatsheet.tex,1
2,capstones,1
3,2-storytelling/chart-types,1
3,2.1-spreadsheets-overview,1
3,3-sql,1


In [154]:
least_df= df.groupby(['program_id', 'path']).agg('count')
least_df

user_id  cohort_id  ip  name  \
program_id path                                                             
4.0        %20https://github.com/RaulCPena         1          1   1     1   
           '                                       1          1   1     1   
           ,%20https://github.com/RaulCPena        1          1   1     1   
           .git                                    1          1   1     1   
           .gitignore                              2          2   2     2   
...                                              ...        ...  ..   ...   
           web-dev-day-two                         2          2   2     2   
           where                                   1          1   1     1   
           working-with-time-series-data          15         15  15    15   
           wp-admin                                1          1   1     1   
           wp-login                                1          1   1     1   

                                             start_date  end_date  created_at  \
program_id path                                                                 
4.0        %20https://github.com/RaulCPena            1         1           1   
           '                                          1         1           1   
           ,%20https://github.com/RaulCPena           1         1           1   
           .git                                       1         1           1   
           .gitignore                                 2         2           2   
...                                                 ...       ...         ...   
           web-dev-day-two                            2         2           2   
           where                                      1         1           1   
           working-with-time-series-data             15        15          15   
           wp-admin                                   1         1           1   
           wp-login                                   1         1           1   

                                             updated_at  data  web  php  \
program_id path                                                           
4.0        %20https://github.com/RaulCPena            1     1    1    1   
           '                                          1     1    1    1   
           ,%20https://github.com/RaulCPena           1     1    1    1   
           .git                                       1     1    1    1   
           .gitignore                                 2     2    2    2   
...                                                 ...   ...  ...  ...   
           web-dev-day-two                            2     2    2    2   
           where                                      1     1    1    1   
           working-with-time-series-data             15    15   15   15   
           wp-admin                                   1     1    1    1   
           wp-login                                   1     1    1    1   

                                             front_end  
program_id path                                         
4.0        %20https://github.com/RaulCPena           1  
           '                                         1  
           ,%20https://github.com/RaulCPena          1  
           .git                                      1  
           .gitignore                                2  
...                                                ...  
           web-dev-day-two                           2  
           where                                     1  
           working-with-time-series-data            15  
           wp-admin                                  1  
           wp-login                                  1  

[2224 rows x 12 columns]

In [145]:
least_df= least_df.sort_values(by= 'user_id', ascending= False)
least_df

user_id  cohort_id     ip   name  \
program_id path                                                             
4.0        /                               45854      45854  45854  45854   
           javascript-i                    18203      18203  18203  18203   
           toc                             17591      17591  17591  17591   
           search/search_index.json        17534      17534  17534  17534   
           java-iii                        13166      13166  13166  13166   
...                                          ...        ...    ...    ...   
           end_to_end_clustering               1          1      1      1   
           essential-methods/traversing        1          1      1      1   
           evaluate                            1          1      1      1   
           examples/bootstrap/pizza.pn         1          1      1      1   
           wp-login                            1          1      1      1   

                                         start_date  end_date  created_at  \
program_id path                                                             
4.0        /                                  45854     45854       45854   
           javascript-i                       18203     18203       18203   
           toc                                17591     17591       17591   
           search/search_index.json           17534     17534       17534   
           java-iii                           13166     13166       13166   
...                                             ...       ...         ...   
           end_to_end_clustering                  1         1           1   
           essential-methods/traversing           1         1           1   
           evaluate                               1         1           1   
           examples/bootstrap/pizza.pn            1         1           1   
           wp-login                               1         1           1   

                                         updated_at   data    web    php  \
program_id path                                                            
4.0        /                                  45854  45854  45854  45854   
           javascript-i                       18203  18203  18203  18203   
           toc                                17591  17591  17591  17591   
           search/search_index.json           17534  17534  17534  17534   
           java-iii                           13166  13166  13166  13166   
...                                             ...    ...    ...    ...   
           end_to_end_clustering                  1      1      1      1   
           essential-methods/traversing           1      1      1      1   
           evaluate                               1      1      1      1   
           examples/bootstrap/pizza.pn            1      1      1      1   
           wp-login                               1      1      1      1   

                                         front_end  
program_id path                                     
4.0        /                                 45854  
           javascript-i                      18203  
           toc                               17591  
           search/search_index.json          17534  
           java-iii                          13166  
...                                            ...  
           end_to_end_clustering                 1  
           essential-methods/traversing          1  
           evaluate                              1  
           examples/bootstrap/pizza.pn           1  
           wp-login                              1  

[2224 rows x 12 columns]

In [146]:
least_df= least_df.reset_index()
least_df

,program_id,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,data,web,php,front_end
0,4.0,/,45854,45854,45854,45854,45854,45854,45854,45854,45854,45854,45854,45854
1,4.0,javascript-i,18203,18203,18203,18203,18203,18203,18203,18203,18203,18203,18203,18203
2,4.0,toc,17591,17591,17591,17591,17591,17591,17591,17591,17591,17591,17591,17591
3,4.0,search/search_index.json,17534,17534,17534,17534,17534,17534,17534,17534,17534,17534,17534,17534
4,4.0,java-iii,13166,13166,13166,13166,13166,13166,13166,13166,13166,13166,13166,13166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,4.0,end_to_end_clustering,1,1,1,1,1,1,1,1,1,1,1,1
2220,4.0,essential-methods/traversing,1,1,1,1,1,1,1,1,1,1,1,1
2221,4.0,evaluate,1,1,1,1,1,1,1,1,1,1,1,1
2222,4.0,examples/bootstrap/pizza.pn,1,1,1,1,1,1,1,1,1,1,1,1
